### Coursework coding instructions (please also see full coursework spec)

Please choose if you want to do either Task 1 or Task 2. You should write your report about one task only.

For the task you choose you will need to do two approaches:
  - Approach 1, which can use use pre-trained embeddings / models
  - Approach 2, which should not use any pre-trained embeddings or models
We should be able to run both approaches from the same colab file

#### Running your code:
  - Your models should run automatically when running your colab file without further intervention
  - For each task you should automatically output the performance of both models
  - Your code should automatically download any libraries required

#### Structure of your code:
  - You are expected to use the 'train', 'eval' and 'model_performance' functions, although you may edit these as required
  - Otherwise there are no restrictions on what you can do in your code

#### Documentation:
  - You are expected to produce a .README file summarising how you have approached both tasks

#### Reproducibility:
  - Your .README file should explain how to replicate the different experiments mentioned in your report

Good luck! We are really looking forward to seeing your reports and your model code!

In [ ]:
# You will need to download any word embeddings required for your code, e.g.:

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

# For any packages that Colab does not provide auotmatically you will also need to install these below, e.g.:

#! pip install torch

--2021-02-19 14:39:25--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-02-19 14:39:25--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-02-19 14:39:25--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
# Download the task dataset

!wget https://www.cs.rochester.edu/u/nhossain/humicroedit/semeval-2020-task-7-data.zip
!unzip semeval-2020-task-7-data.zip
!rm semeval-2020-task-7-data.zip

In [24]:
# Imports

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from torch.utils.data import Dataset, random_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import torch.optim as optim
import codecs
import tqdm

In [25]:
# Setting random seed and device
SEED = 1

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [26]:
# Load data
train_df = pd.read_csv('data/task-2/train.csv')
test_df = pd.read_csv('data/task-2/dev.csv')

In [27]:
# Number of epochs
epochs = 10

# Proportion of training data for train compared to dev
train_proportion = 0.8


#### BERT

##### Make BERT works

#### Approach 1: Using pre-trained representations

In [60]:
# We define our training loop
def train_regression(train_iter, dev_iter, model, number_epoch, input_ids_sentence1, attention_mark_sentence1, input_ids_sentence2, attention_mark_sentence2, dev_dataset, epsilon):
    """
    Training loop for the model, which calls on eval to evaluate after each epoch
    """

    
    print("Training model.")

    for epoch in range(1, number_epoch+1):

        model.train()
        epoch_loss = 0
        epoch_sse = 0
        no_observations = 0  # Observations used for training so far

        for batch in train_iter:

            input_ids, attention_mask, target = batch
            target = target.type(torch.FloatTensor)
            input_ids, attention_mask, target = input_ids.to(device), attention_mask.to(device), target.to(device)
            input_ids = input_ids.squeeze(1)
            attention_mask = attention_mask.squeeze(1)

            model.batch_size = target.shape[0]
            no_observations = no_observations + target.shape[0]
            # for RNN:
            # model.hidden = model.init_hidden()
            predictions = model(input_ids = input_ids, attention_mask = attention_mask).squeeze(1)
            optimizer.zero_grad()

            loss = loss_fn(predictions, target)

            sse, __ = model_performance_regression(predictions.detach().cpu().numpy(), target.detach().cpu().numpy())

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()*target.shape[0]
            epoch_sse += sse
            print('One batch')

        valid_loss, valid_mse, valid_acc, __, __ = eval_regression(dev_iter, model, input_ids_sentence1, attention_mark_sentence1, input_ids_sentence2, attention_mark_sentence2, dev_dataset, epsilon)
        epoch_loss, epoch_mse = epoch_loss / no_observations, epoch_sse / no_observations
        print(f'| Epoch: {epoch:02} | Train Loss: {epoch_loss:.2f} | Train MSE: {epoch_mse:.2f} | Train RMSE: {epoch_mse**0.5:.2f} | \
        Val. Loss: {valid_loss:.2f} | Val. MSE: {valid_mse:.2f} |  Val. RMSE: {valid_mse**0.5:.2f} |  Val. Acc: {valid_acc:.2f} |')

In [29]:
# We define our training loop
def train(train_iter, dev_iter, model, number_epoch):
    """
    Training loop for the model, which calls on eval to evaluate after each epoch
    """

    print("Training model.")

    for epoch in range(1, number_epoch+1):
        
        model.train()
        
        epoch_loss = 0
        epoch_correct = 0
        no_observations = 0  # Observations used for training so far

        for batch in train_iter:
            input_ids, attention_mask, target = batch
            input_ids, attention_mask, target = input_ids.to(device), attention_mask.to(device), target.to(device)

            # for RNN:
            # model.batch_size = target.shape[0]
            no_observations = no_observations + target.shape[0]
            # model.hidden = model.init_hidden()

            # for BERT
            input_ids = input_ids.squeeze(1)
            attention_mask = attention_mask.squeeze(1)
            predictions = model(input_ids = input_ids, attention_mask = attention_mask)
            loss = loss_fn(predictions, target)

            correct, __ = model_performance(np.argmax(predictions.detach().cpu().numpy(), axis=1), target.detach().cpu().numpy())

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()*target.shape[0]
            epoch_correct += correct

        valid_loss, valid_acc, __, __ = eval(dev_iter, model)

        epoch_loss, epoch_acc = epoch_loss / no_observations, epoch_correct / no_observations
        print(f'| Epoch: {epoch:02} | Train Loss: {epoch_loss:.2f} | Train Accuracy: {epoch_acc:.2f} | \
        Val. Loss: {valid_loss:.2f} | Val. Accuracy: {valid_acc:.2f} |')

In [30]:
# We evaluate performance on our dev set
def eval_regression(data_iter, model, input_ids_sentence1, attention_mark_sentence1, input_ids_sentence2, attention_mark_sentence2, dev_dataset, epsilon):
    """
    Evaluating model performance on the dev set
    """
    model.eval()
    epoch_loss = 0
    epoch_sse = 0
    pred_all = []
    trg_all = []
    no_observations = 0

    with torch.no_grad():
        for batch in data_iter:
            input_ids, attention_mask, target = batch
            target = target.type(torch.FloatTensor)
            input_ids, attention_mask, target = input_ids.to(device), attention_mask.to(device), target.to(device)
            input_ids = input_ids.squeeze(1)
            attention_mask = attention_mask.squeeze(1)
            
            model.batch_size = target.shape[0]
            no_observations = no_observations + target.shape[0]
            # for RNN:
            # model.hidden = model.init_hidden()

            predictions = model(input_ids = input_ids, attention_mask = attention_mask).squeeze(1)
            loss = loss_fn(predictions, target)

            # We get the mse
            pred, trg = predictions.detach().cpu().numpy(), target.detach().cpu().numpy()
            sse, __ = model_performance_regression(pred, trg)

            epoch_loss += loss.item()*target.shape[0]
            epoch_sse += sse
            pred_all.extend(pred)
            trg_all.extend(trg)
          
        prediction_data_sentence1, prediction_data_sentence2 = get_predictions(input_ids_sentence1, attention_mark_sentence1, input_ids_sentence2, attention_mark_sentence2)
        acc = get_accuracy(dev_dataset, prediction_data_sentence1, prediction_data_sentence2, epsilon)

    return epoch_loss/no_observations, epoch_sse/no_observations, acc, np.array(pred_all), np.array(trg_all)

In [62]:
def get_predictions(input_ids_sentence1, attention_mark_sentence1, input_ids_sentence2, attention_mark_sentence2):
  batch_size = 64
  model.eval()
  prediction_data1 = []
  prediction_data2 = []
  for i in range(0, len(input_ids_sentence1) // batch_size - 1):
      prediction_data1 += model(input_ids = input_ids_sentence1[i*batch_size:(i+1)*batch_size], attention_mask = attention_mark_sentence1[i*batch_size:(i+1)*batch_size]).detach().squeeze(1).tolist()
      prediction_data2 += model(input_ids = input_ids_sentence2[i*batch_size:(i+1)*batch_size], attention_mask = attention_mark_sentence2[i*batch_size:(i+1)*batch_size]).detach().squeeze(1).tolist()
  prediction_data1 += model(input_ids = input_ids_sentence1[(len(input_ids_sentence1) // batch_size - 1)*batch_size:], attention_mask = attention_mark_sentence1[(len(input_ids_sentence1) // batch_size - 1)*batch_size:]).detach().squeeze(1).tolist()
  prediction_data2 += model(input_ids = input_ids_sentence2[(len(input_ids_sentence1) // batch_size - 1)*batch_size:], attention_mask = attention_mark_sentence2[(len(input_ids_sentence1) // batch_size - 1)*batch_size:]).detach().squeeze(1).tolist()
  return prediction_data1, prediction_data2

In [64]:
def get_accuracy(dataset, predicted1, predicted2, epsilon):
  dataset['predictedGrade1'] = predicted1
  dataset['predictedGrade2'] = predicted2
  label = []
  for i in range(len(predicted1)):
    diff = predicted1[i] - predicted2[i]
    if diff > epsilon:
      label.append(1)
    elif diff < epsilon:
      label.append(2)
    else:
      label.append(0)
  dataset['predictedLabel'] = label
  dataset['correct'] = (dataset['trueLabel'] == dataset['predictedLabel'])
  return dataset['correct'].value_counts()[1] / (dataset['correct'].value_counts()[0] + dataset['correct'].value_counts()[1])

In [33]:
# We evaluate performance on our dev set
def eval(data_iter, model):
    """
    Evaluating model performance on the dev set
    """
    model.eval()
    epoch_loss = 0
    epoch_correct = 0
    pred_all = []
    trg_all = []
    no_observations = 0

    with torch.no_grad():
        for batch in data_iter:
            input_ids, attention_mask, target = batch
            input_ids, attention_mask, target = input_ids.to(device), attention_mask.to(device), target.to(device)

            # for RNN:
            # model.batch_size = target.shape[0]
            no_observations = no_observations + target.shape[0]
            # model.hidden = model.init_hidden()

            # for BERT
            input_ids = input_ids.squeeze(1)
            attention_mask = attention_mask.squeeze(1)
            predictions = model(input_ids = input_ids, attention_mask = attention_mask)
            loss = loss_fn(predictions, target)

            # We get the mse
            pred, trg = predictions.detach().cpu().numpy(), target.detach().cpu().numpy()
            correct, __ = model_performance(np.argmax(pred, axis=1), trg)

            epoch_loss += loss.item()*target.shape[0]
            epoch_correct += correct
            pred_all.extend(pred)
            trg_all.extend(trg)

    return epoch_loss/no_observations, epoch_correct/no_observations, np.array(pred_all), np.array(trg_all)

In [34]:
# How we print the model performance
def model_performance_regression(output, target, print_output=False):
    """
    Returns SSE and MSE per batch (printing the MSE and the RMSE)
    """

    sq_error = (output - target)**2

    sse = np.sum(sq_error)
    mse = np.mean(sq_error)
    rmse = np.sqrt(mse)

    if print_output:
        print(f'| MSE: {mse:.2f} | RMSE: {rmse:.2f} |')

    return sse, mse

In [35]:
# How we print the model performance
def model_performance(output, target, print_output=False):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    correct_answers = (output == target)
    correct = sum(correct_answers)
    acc = np.true_divide(correct,len(output))

    if print_output:
        print(f'| Acc: {acc:.2f} ')

    return correct, acc

In [36]:
# To create our vocab
def create_vocab(data):
    """
    Creating a corpus of all the tokens used
    """
    tokenized_corpus = [] # Let us put the tokenized corpus in a list

    for sentence in data:

        tokenized_sentence = []

        for token in sentence.split(' '): # simplest split is

            tokenized_sentence.append(token)

        tokenized_corpus.append(tokenized_sentence)

    # Create single list of all vocabulary
    vocabulary = []  # Let us put all the tokens (mostly words) appearing in the vocabulary in a list

    for sentence in tokenized_corpus:

        for token in sentence:

            if token not in vocabulary:

                if True:
                    vocabulary.append(token)

    return vocabulary, tokenized_corpus

In [37]:
# Used for collating our observations into minibatches:
def collate_fn_padd(batch):
    '''
    We add padding to our minibatches and create tensors for our model
    '''

    batch_labels = [l for f, l in batch]
    batch_features = [f for f, l in batch]

    batch_features_len = [len(f) for f, l in batch]

    seq_tensor = torch.zeros((len(batch), max(batch_features_len))).long()

    for idx, (seq, seqlen) in enumerate(zip(batch_features, batch_features_len)):
        seq_tensor[idx, :seqlen] = torch.LongTensor(seq)

    batch_labels = torch.LongTensor(batch_labels)

    return seq_tensor, batch_labels

# We create a Dataset so we can create minibatches
class Task1Dataset(Dataset):

    def __init__(self, train_data, labels):
        self.x_train = train_data
        self.y_train = labels

    def __len__(self):
        return len(self.y_train)

    def __getitem__(self, item):
        return self.x_train[item]['input_ids'], self.x_train[item]['attention_mask'], self.y_train[item]

class Task2Dataset(Dataset):

    def __init__(self, tokenized_data, labels):
        self.tokenized_data = tokenized_data
        self.y_train = labels

    def __len__(self):
        return len(self.y_train)

    def __getitem__(self, item):
        return self.tokenized_data[item]['input_ids'], self.tokenized_data[item]['attention_mask'], self.y_train[item]

In [38]:
%pip install transformers
import transformers
from transformers import BertTokenizer, BertPreTrainedModel, BertModel
bert_model = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(bert_model)

In [39]:
class BERT_regression(BertPreTrainedModel):

  def __init__(self, config):
    super().__init__(config)

    # BERT model
    self.bert = BertModel(config)
    
    # Regression layer
    self.final_layer = torch.nn.Sequential(torch.nn.Dropout(0.2),
                                           torch.nn.Linear(config.hidden_size , 1))
    
  def forward(
    self,
    input_ids=None,
    attention_mask=None,
    token_type_ids=None,
    position_ids=None,
    head_mask=None,
    inputs_embeds=None,
    labels=None,
    output_attentions=None,
    output_hidden_states=None,
    return_dict=None):
 
    outputs = self.bert(
        input_ids,
        attention_mask=attention_mask,
        token_type_ids=token_type_ids,
        position_ids=position_ids,
        head_mask=head_mask,
        inputs_embeds=inputs_embeds,
        output_attentions=output_attentions,
        output_hidden_states=output_hidden_states,
        return_dict=return_dict,
    )

    # Regression
    out = self.final_layer(outputs[1])
        
    return out

In [40]:
import re

def replace_word(sentence, new_word):
  search = re.search("<(.*)/>", sentence)
  word_to_replace = "<" + search.group(1) + "/>"
  # sentence = sentence.replace(word_to_replace,  new_word)
  sentence = sentence.replace(word_to_replace, new_word)
  return sentence

In [41]:
def replace_word_dataset(sentence_dataset, word_dataset):
  new_dataset = []
  for i in range (len(sentence_dataset)):
    new_sentence = replace_word(sentence_dataset[i], word_dataset[i])
    new_dataset.append(new_sentence)
  return new_dataset

In [42]:
# We set our training data and test data

training_data1 = train_df['original1']
training_edit1 = train_df['edit1']
test_data1 = test_df['original1']
test_edit1 = test_df['edit1']

training_data2 = train_df['original2']
training_edit2 = train_df['edit2']
test_data2 = test_df['original2']
test_edit2 = test_df['edit2']

# We replace with the editted word
edit_training_data1 = replace_word_dataset(training_data1, training_edit1)
edit_test_data1 = replace_word_dataset(test_data1, test_edit1)
edit_training_data2 = replace_word_dataset(training_data2, training_edit2)
edit_test_data2 = replace_word_dataset(test_data2, test_edit2)

In [43]:
# We create a new dataframe with the editted sentences, their grade and the label

# Get the total dataset

editted_data = {'sentence1': edit_training_data1 ,
        'sentence2': edit_training_data2,
        'meanGrade1': train_df['meanGrade1'].values,
        'meanGrade2': train_df['meanGrade2'].values,
        'trueLabel': train_df['label'].values}

editted_df = pd.DataFrame(editted_data, columns = ['sentence1', 'sentence2', 'meanGrade1', 'meanGrade2', 'trueLabel'])

In [44]:
# We create only one column of sentences for regression

def transform_editted_to_regression(dataset):
  data = {'sentence': [*dataset['sentence1'], *dataset['sentence2']],
        'meanGrade': [*dataset['meanGrade1'], *dataset['meanGrade2']]}

  df = pd.DataFrame(data, columns = ['sentence', 'meanGrade'])
  df = df.drop_duplicates()
  return df

In [52]:
# TODO : work on max length
def tokenized_dataset(dataset):
  tokenized = []
  for i in range(len(dataset)):
    encoding = tokenizer.encode_plus(dataset[i], max_length=30,  return_token_type_ids=False, pad_to_max_length=True, return_attention_mask=True, return_tensors='pt')
    tokenized.append(encoding)
  return tokenized

In [53]:
def get_input_ids_attention_mask(dataset):
  input_ids = []
  attention_mask = []
  tokenized = tokenized_dataset(dataset)
  for i in range(len(dataset)):
    print()
    input_ids.append(tokenized[i]['input_ids'].tolist()[0])
    attention_mask.append(tokenized[i]['attention_mask'].tolist()[0])
  return torch.LongTensor(input_ids).to(device), torch.LongTensor(attention_mask).to(device)

In [65]:
from sklearn.model_selection import train_test_split

BATCH_SIZE = 64
EPSILON = 10**(-4)
model = BERT_regression.from_pretrained(bert_model)
print("Model initialised.")
model.to(device)

# We provide the model with our embeddings
# model.embedding.weight.data.copy_(torch.from_numpy(wvecs))

train_df, dev_df = train_test_split(editted_df, test_size=1 - train_proportion)

regression_train_df = transform_editted_to_regression(train_df)
regression_dev_df = transform_editted_to_regression(dev_df)

train_feature = tokenized_dataset(regression_train_df['sentence'].values)
dev_feature = tokenized_dataset(regression_dev_df['sentence'].values)

# 'feature' is a list of lists, each containing embedding IDs for word tokens
train_dataset = Task1Dataset(train_feature, regression_train_df['meanGrade'].values)
dev_dataset = Task1Dataset(dev_feature, regression_dev_df['meanGrade'].values)

# train_examples_regression = round(len(train_and_dev)*train_proportion)
# dev_examples_regression = len(train_and_dev) - train_examples

# train_dataset, dev_dataset = random_split(train_and_dev,
#                                            (train_examples,
#                                             dev_examples))


train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE) #, collate_fn=collate_fn_padd)
dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=BATCH_SIZE) #, collate_fn=collate_fn_padd)

print("Dataloaders created.")

loss_fn = nn.MSELoss()
loss_fn = loss_fn.to(device)

# optimizer = torch.optim.Adam(model.parameters())
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Get element to test result on dev_dataset
dev_sentence1_input_ids, dev_sentence1_attention_mask = get_input_ids_attention_mask(dev_df['sentence1'].values)
dev_sentence2_input_ids, dev_sentence2_attention_mask = get_input_ids_attention_mask(dev_df['sentence2'].values)

train_regression(train_loader, dev_loader, model, epochs, dev_sentence1_input_ids, dev_sentence1_attention_mask, dev_sentence2_input_ids, dev_sentence2_attention_mask, dev_df, EPSILON)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BERT_regression: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BERT_regression from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BERT_regression from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BERT_regression were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['final_layer.

Model initialised.
Dataloaders created.


































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

UnboundLocalError: ignored

In [ ]:
BATCH_SIZE = 64

import transformers
from transformers import BertForSequenceClassification
# We initialise BERT model for classification

model = BERT_regression.from_pretrained(bert_model, num_labels=3)
print("Model initialised.")
model.to(device)

# 'feature' is a list of lists, each containing embedding IDs for word tokens
train_and_dev = Task2Dataset(tokenized_training, train_df['label'])

train_examples = round(len(train_and_dev)*train_proportion)
dev_examples = len(train_and_dev) - train_examples

train_dataset, dev_dataset = random_split(train_and_dev,
                                           (train_examples,
                                            dev_examples))


train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE) #, collate_fn=collate_fn_padd)
dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=BATCH_SIZE) #, collate_fn=collate_fn_padd)

print("Dataloaders created.")

loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)

# optimizer = torch.optim.Adam(model.parameters())
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

train(train_loader, dev_loader, model, epochs)

#### Approach 2: No pre-trained representations

In [ ]:

class BiLSTM_classification(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, batch_size, device):
        super(BiLSTM_classification, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.device = device
        self.batch_size = batch_size
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2label = nn.Linear(hidden_dim * 2, 3)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly why they have this dimensionality.
        # The axes semantics are (num_layers * num_directions, minibatch_size, hidden_dim)
        return torch.zeros(2, self.batch_size, self.hidden_dim).to(self.device), \
               torch.zeros(2, self.batch_size, self.hidden_dim).to(self.device)

    def forward(self, sentence):
        embedded = self.embedding(sentence)
        embedded = embedded.permute(1, 0, 2)

        lstm_out, self.hidden = self.lstm(
            embedded.view(len(embedded), self.batch_size, self.embedding_dim), self.hidden)

        out = self.hidden2label(lstm_out[-1])
        return out

In [ ]:
train_and_dev = train_df['edit1']

training_data, dev_data, training_y, dev_y = train_test_split(train_df['edit1'], train_df['label'],
                                                                        test_size=(1-train_proportion),
                                                                        random_state=42)

# We train a Tf-idf model
count_vect = CountVectorizer(stop_words='english')
train_counts = count_vect.fit_transform(training_data)
transformer = TfidfTransformer().fit(train_counts)
train_counts = transformer.transform(train_counts)
naive_model = MultinomialNB().fit(train_counts, training_y)

# Train predictions
predicted_train = naive_model.predict(train_counts)

# Calculate Tf-idf using train and dev, and validate on dev:
test_and_test_counts = count_vect.transform(train_and_dev)
transformer = TfidfTransformer().fit(test_and_test_counts)

test_counts = count_vect.transform(dev_data)

test_counts = transformer.transform(test_counts)

# Dev predictions
predicted = naive_model.predict(test_counts)

# We run the evaluation:
print("\nTrain performance:")

sse, mse = model_performance(predicted_train, training_y, True)

print("\nDev performance:")
sse, mse = model_performance(predicted, dev_y, True)

#### Baseline for task 2

In [ ]:
# Baseline for the task
pred_baseline = torch.zeros(len(dev_y)) + 1  # 1 is most common class
print("\nBaseline performance:")
sse, mse = model_performance(pred_baseline, torch.tensor(dev_y.values), True)